In [8]:
import pandas as pd 

from utils.preprocessing import preprocess_df
from utils.df_loader import load_adult_df, load_compas_df, load_german_df, load_diabetes_df
from utils.evaluation import get_evaluations, EvaluationMatrix

In [9]:
#### Select dataset ####
dataset_name = 'adult' # [adult, german, compas]

if dataset_name == 'adult':
    dataset_loading_fn = load_adult_df
elif dataset_name == 'german':
    dataset_loading_fn = load_german_df
elif dataset_name == 'compas':
    dataset_loading_fn = load_compas_df
elif dataset_name == 'diabetes':
    dataset_loading_fn = load_diabetes_df
else:
    raise Exception("Unsupported dataset")

df_info = preprocess_df(dataset_loading_fn)

/Users/jrhs/Documents/GitHub/Counterfactual-benchmark/utils/preprocessing.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][df[col] == '?'] = df[col].value_counts().index[0]


In [10]:
folder_name = 'proto_adult'
model_name = 'rfc'
file_name = f'{folder_name}_{model_name}_result.csv'
result_path = f'./results/{folder_name}/{file_name}'
result_df = pd.read_csv(result_path)

In [11]:
result_df

,Unnamed: 0,scaled_input_age,scaled_input_capital-gain,scaled_input_capital-loss,scaled_input_hours-per-week,scaled_input_workclass,scaled_input_education,scaled_input_marital-status,scaled_input_occupation,scaled_input_relationship,...,origin_cf_hours-per-week,origin_cf_workclass,origin_cf_education,origin_cf_marital-status,origin_cf_occupation,origin_cf_relationship,origin_cf_race,origin_cf_sex,origin_cf_native-country,origin_cf_class
0,0,0.520548,0.0,0.0,0.142857,State-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0.520548,0.0,0.0,0.142857,State-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0.520548,0.0,0.0,0.142857,State-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0.520548,0.0,0.0,0.142857,State-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0.520548,0.0,0.0,0.142857,State-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0.068493,0.0,0.0,0.397959,Private,HS-grad,Widowed,Adm-clerical,Own-child,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,0,0.068493,0.0,0.0,0.397959,Private,HS-grad,Widowed,Adm-clerical,Own-child,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,0,0.068493,0.0,0.0,0.397959,Private,HS-grad,Widowed,Adm-clerical,Own-child,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,0,0.068493,0.0,0.0,0.397959,Private,HS-grad,Widowed,Adm-clerical,Own-child,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
evaluation_df = get_evaluations(result_df, df_info, matrix = [EvaluationMatrix.L1, EvaluationMatrix.L2, EvaluationMatrix.Sparsity, EvaluationMatrix.Realistic, EvaluationMatrix.MAD, EvaluationMatrix.Mahalanobis])

In [14]:
evaluation_df[evaluation_df['Found']=="Y"]

,Unnamed: 0,scaled_input_age,scaled_input_capital-gain,scaled_input_capital-loss,scaled_input_hours-per-week,scaled_input_workclass,scaled_input_education,scaled_input_marital-status,scaled_input_occupation,scaled_input_relationship,...,origin_cf_race,origin_cf_sex,origin_cf_native-country,origin_cf_class,L1,L2,Sparsity,Realistic,MAD,Mahalanobis
15,0,0.493151,0.000000,0.000000,0.397959,State-gov,Doctorate,Married-civ-spouse,Prof-specialty,Husband,...,White,Female,Haiti,>50K,10.891110,3.225146,12.0,True,3.091775,1.486033
16,0,0.493151,0.000000,0.000000,0.397959,State-gov,Doctorate,Married-civ-spouse,Prof-specialty,Husband,...,White,Female,Haiti,>50K,10.891110,3.225146,12.0,True,3.091775,1.486033
17,0,0.493151,0.000000,0.000000,0.397959,State-gov,Doctorate,Married-civ-spouse,Prof-specialty,Husband,...,White,Female,Haiti,>50K,10.891110,3.225146,12.0,True,3.091775,1.486033
18,0,0.493151,0.000000,0.000000,0.397959,State-gov,Doctorate,Married-civ-spouse,Prof-specialty,Husband,...,White,Female,Haiti,>50K,10.891110,3.225146,12.0,True,3.091775,1.486033
19,0,0.493151,0.000000,0.000000,0.397959,State-gov,Doctorate,Married-civ-spouse,Prof-specialty,Husband,...,White,Female,Haiti,>50K,10.891110,3.225146,12.0,True,3.091775,1.486033
30,0,0.205479,0.000000,0.000000,0.428571,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,...,White,Female,Haiti,>50K,12.634051,3.496555,14.0,True,NaN,1.604152
31,0,0.205479,0.000000,0.000000,0.428571,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,...,White,Female,Haiti,>50K,12.634051,3.496555,14.0,True,NaN,1.604152
32,0,0.205479,0.000000,0.000000,0.428571,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,...,White,Female,Haiti,>50K,12.634051,3.496555,14.0,True,NaN,1.604152
33,0,0.205479,0.000000,0.000000,0.428571,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,...,White,Female,Haiti,>50K,12.634051,3.496555,14.0,True,NaN,1.604152
34,0,0.205479,0.000000,0.000000,0.428571,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,...,White,Female,Haiti,>50K,12.634051,3.496555,14.0,True,NaN,1.604152


In [7]:
csv_save_result_path = f'results/{folder_name}/eval_{file_name}'
evaluation_df.to_csv(csv_save_result_path)